In [19]:
import pandas as pd
import sqlite3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import optuna
import os
import json 


conn = sqlite3.connect(r'C:\Users\Owner\dev\algobetting\infra\data\db\algobetting.db')

df = pd.read_sql_query("""
    SELECT DISTINCT
        f.match_date,
        f.team,
        f.is_home,
        f.gw,
        f.[is_promoted?],
        f.[is_early_season?],
        f.season,
        f.division,
        team_elo.elo as team_elo,
        f.team_rolling_summary_goals,
        f.team_rolling_summary_pens_made,
        f.team_rolling_summary_shots,
        f.team_rolling_summary_shots_on_target,
        f.team_rolling_summary_touches,
        f.team_rolling_summary_xg,
        f.team_rolling_summary_npxg,
        f.team_rolling_possession_touches_att_pen_area,
        f.team_rolling_keeper_psxg,
        f.team_rolling_conceded_summary_goals,
        f.team_rolling_conceded_summary_pens_made,
        f.team_rolling_conceded_summary_shots,
        f.team_rolling_conceded_summary_shots_on_target,
        f.team_rolling_conceded_summary_touches,
        f.team_rolling_conceded_summary_xg,
        f.team_rolling_conceded_summary_npxg,
        f.team_rolling_conceded_possession_touches_att_pen_area,
        f.team_rolling_conceded_keeper_psxg,
        f.opp_team,
        opp_elo.elo as opp_team_elo,
        f.opp_team_rolling_summary_goals,
        f.opp_team_rolling_summary_pens_made,
        f.opp_team_rolling_summary_shots,
        f.opp_team_rolling_summary_shots_on_target,
        f.opp_team_rolling_summary_touches,
        f.opp_team_rolling_summary_xg,
        f.opp_team_rolling_summary_npxg,
        f.opp_team_rolling_possession_touches_att_pen_area,
        f.opp_team_rolling_keeper_psxg,
        f.opp_team_rolling_conceded_summary_goals,
        f.opp_team_rolling_conceded_summary_pens_made,
        f.opp_team_rolling_conceded_summary_shots,
        f.opp_team_rolling_conceded_summary_shots_on_target,
        f.opp_team_rolling_conceded_summary_touches,
        f.opp_team_rolling_conceded_summary_xg,
        f.opp_team_rolling_conceded_summary_npxg,
        f.opp_team_rolling_conceded_possession_touches_att_pen_area,
        f.opp_team_rolling_conceded_keeper_psxg,
        ms.summary_goals as goals
    FROM 
        team_all_features_90_00200 f
    JOIN
        fbref_match_all_columns ms
            ON ms.match_url = f.match_url
            AND ms.team = f.team
    JOIN
        clubelo_features team_elo
            ON team_elo.fbref_team = f.team
                AND DATE(f.match_date) BETWEEN team_elo.start_date AND team_elo.end_date
    JOIN
        clubelo_features opp_elo
            ON opp_elo.fbref_team = f.opp_team
                AND DATE(f.match_date) BETWEEN opp_elo.start_date AND opp_elo.end_date
    WHERE 
        f.team_rolling_summary_goals IS NOT NULL
        AND f.division = 'Premier League'
                       """, conn)

df

,match_date,team,is_home,gw,is_promoted?,is_early_season?,season,division,team_elo,team_rolling_summary_goals,...,opp_team_rolling_conceded_summary_goals,opp_team_rolling_conceded_summary_pens_made,opp_team_rolling_conceded_summary_shots,opp_team_rolling_conceded_summary_shots_on_target,opp_team_rolling_conceded_summary_touches,opp_team_rolling_conceded_summary_xg,opp_team_rolling_conceded_summary_npxg,opp_team_rolling_conceded_possession_touches_att_pen_area,opp_team_rolling_conceded_keeper_psxg,goals
0,2025-05-25 00:00:00,Tottenham,1,38,0,0,2024-2025,Premier League,1778.323242,0.811222,...,1.604323,0.189265,12.478522,4.128437,603.795044,1.482446,1.331035,26.528746,1.392191,1.0
1,2025-05-25 00:00:00,Brighton,0,38,0,0,2024-2025,Premier League,1809.790894,1.942928,...,2.199083,0.000000,15.406190,5.449610,593.018515,1.694755,1.694755,26.197995,2.080640,4.0
2,2025-05-25 00:00:00,Bournemouth,1,38,0,0,2024-2025,Premier League,1806.686401,1.221826,...,1.501753,0.165470,17.459047,4.194789,654.869872,1.860305,1.690205,28.673113,1.399942,2.0
3,2025-05-25 00:00:00,Leicester City,0,38,1,0,2024-2025,Premier League,1620.328247,1.052791,...,1.662430,0.099022,11.401362,4.082394,605.052197,1.304388,1.225170,23.430866,1.373387,0.0
4,2025-05-25 00:00:00,Newcastle Utd,1,38,0,0,2024-2025,Premier League,1878.094604,1.851561,...,0.968051,0.000000,11.809791,4.587958,650.468474,1.138399,1.138399,22.299200,1.289435,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5330,2020-09-12 00:00:00,Leeds United,0,1,1,1,2020-2021,Premier League,1624.624268,2.418047,...,1.436920,0.095730,7.266258,2.635221,496.180466,1.205448,1.128864,10.108849,1.160007,3.0
5331,2020-09-12 00:00:00,Crystal Palace,1,1,0,1,2020-2021,Premier League,1673.688843,0.570460,...,0.822600,0.000000,10.651285,3.048056,544.952012,1.216448,1.216448,20.837488,1.270361,1.0
5332,2020-09-12 00:00:00,Southampton,0,1,0,1,2020-2021,Premier League,1720.211182,1.923482,...,1.970332,0.000000,12.669481,4.397582,686.387950,1.478836,1.478836,20.791894,1.788755,0.0
5333,2020-09-12 00:00:00,Fulham,1,1,1,1,2020-2021,Premier League,1591.546265,1.364007,...,1.193816,0.199407,13.800604,4.952256,574.073749,1.695469,1.538354,24.615318,1.463729,0.0


In [20]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import poisson

def poisson_log_likelihood(y_true, y_pred):
    """
    Calculate log-likelihood for Poisson distribution
    Higher values are better (less negative)
    """
    y_pred = np.maximum(y_pred, 1e-10)  # Avoid log(0)
    return np.sum(poisson.logpmf(y_true, y_pred))

def poisson_deviance(y_true, y_pred):
    """
    Calculate Poisson deviance - lower is better
    """
    y_pred = np.maximum(y_pred, 1e-10)
    y_true_safe = np.maximum(y_true, 1e-10)
    
    deviance = 2 * np.sum(
        y_true * np.log(y_true_safe / y_pred) - (y_true - y_pred)
    )
    return deviance

def calibration_analysis(y_true, y_pred, n_bins=10):
    """
    Check if predicted rates match observed rates across different ranges
    """
    bin_edges = np.percentile(y_pred, np.linspace(0, 100, n_bins + 1))
    bin_indices = np.digitize(y_pred, bin_edges) - 1
    bin_indices = np.clip(bin_indices, 0, n_bins - 1)
    
    print(f"\n=== CALIBRATION ANALYSIS ===")
    print("Bin | Predicted | Observed | Count | Abs Diff")
    print("-" * 45)
    
    total_abs_diff = 0
    for i in range(n_bins):
        mask = bin_indices == i
        if np.sum(mask) > 0:
            pred_mean = y_pred[mask].mean()
            obs_mean = y_true[mask].mean()
            count = np.sum(mask)
            abs_diff = abs(pred_mean - obs_mean)
            total_abs_diff += abs_diff
            print(f"{i+1:2d}  | {pred_mean:8.3f} | {obs_mean:7.3f} | {count:4d} | {abs_diff:7.3f}")
    
    mean_abs_calibration_error = total_abs_diff / n_bins
    print(f"\nMean Absolute Calibration Error: {mean_abs_calibration_error:.4f}")
    return mean_abs_calibration_error

# Your existing preprocessing code
X = df.drop(columns=["team", "opp_team", "goals", "match_date"])
X = pd.get_dummies(X, columns=["season", "division"], drop_first=True)
y = df["goals"]

# Train/test split
cutoff_date = '2024-08-01'
df['match_date'] = pd.to_datetime(df['match_date'])

train_mask = df['match_date'] < cutoff_date
test_mask = df['match_date'] >= cutoff_date

X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Features: {len(X.columns)}")

# Create DMatrix objects
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# XGBoost parameters
params = {
    'objective': 'count:poisson',
    'max_depth': 7,
    'eta': 0.1,
    'subsample': 0.7,
    'colsample_bytree': 0.8,
    'eval_metric': 'poisson-nloglik'
}

num_rounds = 100
eval_list = [(dtrain, 'train'), (dtest, 'eval')]

# Train model
print("\nTraining XGBoost Poisson model...")
model = xgb.train(
    params, 
    dtrain, 
    num_rounds, 
    eval_list, 
    early_stopping_rounds=10,
    verbose_eval=10
)

# Make predictions
preds = model.predict(dtest)

# Calculate ALL metrics
rmse = np.sqrt(mean_squared_error(y_test, preds))
mae = mean_absolute_error(y_test, preds)

# Poisson-specific metrics
log_likelihood = poisson_log_likelihood(y_test, preds)
mean_log_likelihood = log_likelihood / len(y_test)
deviance = poisson_deviance(y_test, preds)

# AIC for model comparison
n_params = len(X.columns) + 1  # features + intercept
aic = 2 * n_params - 2 * log_likelihood

print(f"\n=== FINAL MODEL EVALUATION ===")
print(f"Traditional Metrics:")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE: {mae:.4f}")

print(f"\nPoisson-Specific Metrics (PRIMARY):")
print(f"  Log-likelihood: {log_likelihood:.2f}")
print(f"  Mean log-likelihood: {mean_log_likelihood:.4f}")
print(f"  Poisson deviance: {deviance:.2f}")
print(f"  AIC: {aic:.2f}")

print(f"\nPrediction Summary:")
print(f"  Mean prediction: {preds.mean():.3f}")
print(f"  Mean actual: {y_test.mean():.3f}")
print(f"  Prediction std: {preds.std():.3f}")
print(f"  Actual std: {y_test.std():.3f}")

# Calibration analysis
calibration_error = calibration_analysis(y_test, preds)

# Additional diagnostic: prediction distribution
print(f"\n=== PREDICTION DISTRIBUTION ===")
pred_percentiles = [5, 10, 25, 50, 75, 90, 95]
actual_percentiles = [5, 10, 25, 50, 75, 90, 95]

print("Percentile | Predicted | Actual")
print("-" * 30)
for p in pred_percentiles:
    pred_p = np.percentile(preds, p)
    actual_p = np.percentile(y_test, p)
    print(f"{p:8d}th | {pred_p:8.3f} | {actual_p:5.0f}")

# Feature importance (top 15)
print(f"\n=== TOP 15 FEATURE IMPORTANCE ===")
importance = model.get_score(importance_type='weight')
sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)

for i, (feature, score) in enumerate(sorted_importance[:15]):
    print(f"{i+1:2d}. {feature:<40} {score:>6}")

print(f"\n=== MODEL QUALITY SUMMARY ===")
print(f"✓ Log-likelihood: {log_likelihood:.2f} (higher = better)")
print(f"✓ Mean calibration error: {calibration_error:.4f} (lower = better)")
print(f"✓ Mean prediction vs actual: {preds.mean():.3f} vs {y_test.mean():.3f}")
if calibration_error < 0.1:
    print("✅ Model is well-calibrated (calibration error < 0.1)")
else:
    print("⚠️  Model may need calibration improvement")

Training samples: 4673
Test samples: 662
Features: 49

Training XGBoost Poisson model...
[0]	train-poisson-nloglik:1.55238	eval-poisson-nloglik:1.56782
[10]	train-poisson-nloglik:1.43734	eval-poisson-nloglik:1.52596
[20]	train-poisson-nloglik:1.37391	eval-poisson-nloglik:1.50811


c:\Users\Owner\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[30]	train-poisson-nloglik:1.32942	eval-poisson-nloglik:1.50475
[39]	train-poisson-nloglik:1.29982	eval-poisson-nloglik:1.50752

=== FINAL MODEL EVALUATION ===
Traditional Metrics:
  RMSE: 1.2025
  MAE: 0.9382

Poisson-Specific Metrics (PRIMARY):
  Log-likelihood: -998.04
  Mean log-likelihood: -1.5076
  Poisson deviance: 743.25
  AIC: 2096.08

Prediction Summary:
  Mean prediction: 1.450
  Mean actual: 1.486
  Prediction std: 0.435
  Actual std: 1.254

=== CALIBRATION ANALYSIS ===
Bin | Predicted | Observed | Count | Abs Diff
---------------------------------------------
 1  |    0.816 |   0.955 |   67 |   0.139
 2  |    0.979 |   1.061 |   66 |   0.082
 3  |    1.111 |   1.197 |   66 |   0.086
 4  |    1.247 |   1.152 |   66 |   0.095
 5  |    1.358 |   1.455 |   66 |   0.096
 6  |    1.469 |   1.409 |   66 |   0.060
 7  |    1.584 |   1.667 |   66 |   0.082
 8  |    1.728 |   1.864 |   66 |   0.136
 9  |    1.909 |   2.076 |   66 |   0.167
10  |    2.291 |   2.030 |   67 |   0.261



In [21]:
# Get feature importance
importance = model.get_score(importance_type='gain')
feature_names = X_train.columns.tolist()

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({
    'feature': list(importance.keys()),
    'importance': list(importance.values())
}).sort_values('importance', ascending=False)

print("Top 10 Most Important Features:")
print(importance_df.head(15))

Top 10 Most Important Features:
                                              feature  importance
4                                            team_elo    7.198771
23                                       opp_team_elo    5.942600
35            opp_team_rolling_conceded_summary_shots    4.041413
39             opp_team_rolling_conceded_summary_npxg    3.872956
12       team_rolling_possession_touches_att_pen_area    3.786753
40  opp_team_rolling_conceded_possession_touches_a...    3.338941
38               opp_team_rolling_conceded_summary_xg    3.217417
13                           team_rolling_keeper_psxg    3.093684
11                          team_rolling_summary_npxg    2.994737
27           opp_team_rolling_summary_shots_on_target    2.975440
32                       opp_team_rolling_keeper_psxg    2.967831
30                      opp_team_rolling_summary_npxg    2.817076
18              team_rolling_conceded_summary_touches    2.805302
36  opp_team_rolling_conceded_summary_shots_

# Hyperparameter Tuning

In [22]:
# Set to True to tune new parameters, False to load existing
TUNE_NEW_PARAMS = True  # Change this to False to load saved params

# Tuning settings
N_TRIALS = 250
PARAMS_FILE = "best_xgb_params_prem_simple.json"

In [23]:
import optuna
import json
import os
import numpy as np
import xgboost as xgb
from scipy.stats import poisson

if TUNE_NEW_PARAMS or not os.path.exists(PARAMS_FILE):
    print("Tuning new hyperparameters for Poisson regression...")
    
    def objective(trial):
        # Optimized search space for Poisson count data with ~6k rows
        params = {
            'objective': 'count:poisson',
            'eval_metric': 'poisson-nloglik',
            'random_state': 42,
            
            # Tree structure - conservative for count data
            'max_depth': trial.suggest_int('max_depth', 3, 7),  # Slightly deeper for interactions
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),  # Higher for Poisson
            'max_leaves': trial.suggest_int('max_leaves', 0, 63),  # 0 means no limit
            
            # Learning parameters
            'eta': trial.suggest_float('eta', 0.01, 0.3, log=True),  # Wider range
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),  # More aggressive sampling OK
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
            'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 1.0),  # Additional sampling
            
            # Regularization - important for count data
            'gamma': trial.suggest_float('gamma', 0, 10),  # Minimum loss reduction
            'alpha': trial.suggest_float('alpha', 0, 20),  # L1 regularization
            'lambda': trial.suggest_float('lambda', 0.1, 20),  # L2 regularization
            
            # Poisson-specific parameters
            'max_delta_step': trial.suggest_float('max_delta_step', 0, 10),  # Helps with Poisson
        }
        
        n_estimators = trial.suggest_int('n_estimators', 50, 500)
        
        try:
            # Train model
            dtrain = xgb.DMatrix(X_train, label=y_train)
            dtest = xgb.DMatrix(X_test, label=y_test)
            
            model = xgb.train(
                params,
                dtrain,
                num_boost_round=n_estimators,
                evals=[(dtrain, 'train'), (dtest, 'eval')],
                early_stopping_rounds=15,  # Slightly more patience
                verbose_eval=False
            )
            
            # Make predictions and calculate PROPER Poisson log-likelihood
            preds = model.predict(dtest)
            preds = np.maximum(preds, 1e-10)  # Avoid numerical issues
            
            # Calculate negative log-likelihood (what we want to minimize)
            # Using scipy for numerical stability
            log_likelihood = np.sum(poisson.logpmf(y_test, preds))
            neg_log_likelihood = -log_likelihood
            
            # Optionally add penalty for extreme predictions to encourage stability
            pred_penalty = np.mean(np.where(preds > 10, (preds - 10) * 0.1, 0))
            
            return neg_log_likelihood + pred_penalty
            
        except Exception as e:
            # Return a large penalty for failed trials
            return float('inf')
    
    # Create study with TPE sampler and pruning for efficiency
    study = optuna.create_study(
        direction='minimize', 
        sampler=optuna.samplers.TPESampler(
            seed=42,
            n_startup_trials=20,  # Random trials before TPE kicks in
            n_ei_candidates=24,   # More candidates for better exploration
        ),
        pruner=optuna.pruners.MedianPruner(
            n_startup_trials=10,
            n_warmup_steps=20,
            interval_steps=10
        )
    )
    
    print(f"Starting optimization with {N_TRIALS} trials...")
    study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True, n_jobs=1)
    
    # Get best parameters
    best_params = study.best_params.copy()
    n_estimators = best_params.pop('n_estimators')
    
    # Calculate additional metrics for the best model
    print("\nEvaluating best model...")
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    best_model = xgb.train(
        best_params,
        dtrain,
        num_boost_round=n_estimators,
        evals=[(dtrain, 'train'), (dtest, 'eval')],
        early_stopping_rounds=15,
        verbose_eval=False
    )
    
    best_preds = best_model.predict(dtest)
    best_preds = np.maximum(best_preds, 1e-10)
    
    # Calculate comprehensive metrics
    from sklearn.metrics import mean_squared_error, mean_absolute_error
    
    log_likelihood = np.sum(poisson.logpmf(y_test, best_preds))
    mean_log_likelihood = log_likelihood / len(y_test)
    rmse = np.sqrt(mean_squared_error(y_test, best_preds))
    mae = mean_absolute_error(y_test, best_preds)
    
    # Poisson deviance
    y_test_safe = np.maximum(y_test, 1e-10)
    deviance = 2 * np.sum(y_test * np.log(y_test_safe / best_preds) - (y_test - best_preds))
    
    print(f"\n=== OPTIMIZATION RESULTS ===")
    print(f"Best Negative Log-Likelihood: {study.best_value:.4f}")
    print(f"Log-Likelihood: {log_likelihood:.2f}")
    print(f"Mean Log-Likelihood: {mean_log_likelihood:.4f}")
    print(f"Poisson Deviance: {deviance:.2f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    
    print(f"\nBest parameters:")
    for key, value in best_params.items():
        if key not in ['objective', 'eval_metric', 'random_state']:
            print(f"  {key}: {value}")
    print(f"  n_estimators: {n_estimators}")
    
    # Save comprehensive results
    params_to_save = {
        'best_params': best_params,
        'n_estimators': n_estimators,
        'optimization_results': {
            'best_neg_log_likelihood': float(study.best_value),
            'log_likelihood': float(log_likelihood),
            'mean_log_likelihood': float(mean_log_likelihood),
            'poisson_deviance': float(deviance),
            'rmse': float(rmse),
            'mae': float(mae)
        },
        'study_stats': {
            'n_trials': len(study.trials),
            'n_complete_trials': len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]),
            'n_pruned_trials': len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])
        }
    }
    
    with open(PARAMS_FILE, 'w') as f:
        json.dump(params_to_save, f, indent=2)
    print(f"\nParameters saved to {PARAMS_FILE}")
    
    # Optional: Print feature importance of best model
    try:
        importance = best_model.get_score(importance_type='weight')
        sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)
        print(f"\n=== TOP 10 FEATURES (Best Model) ===")
        for i, (feature, score) in enumerate(sorted_importance[:10]):
            print(f"{i+1:2d}. {feature:<35} {score:>6}")
    except:
        pass
    
else:
    print("Loading saved hyperparameters...")
    
    with open(PARAMS_FILE, 'r') as f:
        saved_params = json.load(f)
    
    best_params = saved_params['best_params']
    n_estimators = saved_params['n_estimators']
    
    print("Loaded parameters:")
    for key, value in best_params.items():
        if key not in ['objective', 'eval_metric', 'random_state']:
            print(f"  {key}: {value}")
    print(f"  n_estimators: {n_estimators}")
    
    # Display saved metrics if available
    if 'optimization_results' in saved_params:
        results = saved_params['optimization_results']
        print(f"\nPrevious optimization results:")
        print(f"  Log-likelihood: {results.get('log_likelihood', 'N/A')}")
        print(f"  Mean log-likelihood: {results.get('mean_log_likelihood', 'N/A')}")
        print(f"  RMSE: {results.get('rmse', 'N/A')}")
        print(f"  MAE: {results.get('mae', 'N/A')}")
    
    if 'study_stats' in saved_params:
        stats = saved_params['study_stats']
        print(f"\nOptimization stats:")
        print(f"  Total trials: {stats.get('n_trials', 'N/A')}")
        print(f"  Completed trials: {stats.get('n_complete_trials', 'N/A')}")
        print(f"  Pruned trials: {stats.get('n_pruned_trials', 'N/A')}")

[I 2025-08-06 08:24:08,291] A new study created in memory with name: no-name-074b0dc0-bc51-4243-90a2-f5acdc95526e


Tuning new hyperparameters for Poisson regression...
Starting optimization with 250 trials...


  0%|          | 0/250 [00:00<?, ?it/s]

[I 2025-08-06 08:24:08,701] Trial 0 finished with value: 1094.015585339986 and parameters: {'max_depth': 4, 'min_child_weight': 15, 'max_leaves': 46, 'eta': 0.07661100707771368, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'colsample_bylevel': 0.5290418060840998, 'colsample_bynode': 0.9330880728874675, 'gamma': 6.011150117432088, 'alpha': 14.161451555920909, 'lambda': 0.5096314364864687, 'max_delta_step': 9.699098521619943, 'n_estimators': 425}. Best is trial 0 with value: 1094.015585339986.
[I 2025-08-06 08:24:09,129] Trial 1 finished with value: 1073.9453112111778 and parameters: {'max_depth': 4, 'min_child_weight': 3, 'max_leaves': 11, 'eta': 0.028145092716060652, 'subsample': 0.762378215816119, 'colsample_bytree': 0.7159725093210578, 'colsample_bylevel': 0.645614570099021, 'colsample_bynode': 0.8059264473611898, 'gamma': 1.3949386065204183, 'alpha': 5.842892970704363, 'lambda': 7.390600681544464, 'max_delta_step': 4.56069984217036, 'n_estimators': 404}. 

In [27]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import poisson

def poisson_log_likelihood(y_true, y_pred):
    """Calculate log-likelihood for Poisson distribution"""
    y_pred = np.maximum(y_pred, 1e-10)
    return np.sum(poisson.logpmf(y_true, y_pred))

def poisson_deviance(y_true, y_pred):
    """Calculate Poisson deviance - lower is better"""
    y_pred = np.maximum(y_pred, 1e-10)
    y_true_safe = np.maximum(y_true, 1e-10)
    deviance = 2 * np.sum(y_true * np.log(y_true_safe / y_pred) - (y_true - y_pred))
    return deviance

def calibration_analysis(y_true, y_pred, n_bins=10):
    """Analyze calibration across prediction ranges"""
    bin_edges = np.percentile(y_pred, np.linspace(0, 100, n_bins + 1))
    bin_indices = np.digitize(y_pred, bin_edges) - 1
    bin_indices = np.clip(bin_indices, 0, n_bins - 1)
    
    print(f"\n=== CALIBRATION ANALYSIS ===")
    print("Bin | Predicted | Observed | Count | Abs Diff")
    print("-" * 45)
    
    total_abs_diff = 0
    valid_bins = 0
    
    for i in range(n_bins):
        mask = bin_indices == i
        if np.sum(mask) > 0:
            pred_mean = y_pred[mask].mean()
            obs_mean = y_true[mask].mean()
            count = np.sum(mask)
            abs_diff = abs(pred_mean - obs_mean)
            total_abs_diff += abs_diff
            valid_bins += 1
            print(f"{i+1:2d}  | {pred_mean:8.3f} | {obs_mean:7.3f} | {count:4d} | {abs_diff:7.3f}")
    
    mean_abs_calibration_error = total_abs_diff / valid_bins if valid_bins > 0 else 0
    print(f"\nMean Absolute Calibration Error: {mean_abs_calibration_error:.4f}")
    return mean_abs_calibration_error

print("\nTraining final model with optimized parameters...")

# Prepare data
dtrain_final = xgb.DMatrix(X_train, label=y_train)
dtest_final = xgb.DMatrix(X_test, label=y_test)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Features: {len(X_train.columns) if hasattr(X_train, 'columns') else 'N/A'}")

# Train with best parameters
final_model = xgb.train(
    best_params,
    dtrain_final,
    num_boost_round=n_estimators,
    evals=[(dtrain_final, 'train'), (dtest_final, 'eval')],
    early_stopping_rounds=20,
    verbose_eval=10
)

print(f"\nActual boosting rounds used: {final_model.best_iteration + 1}")

# Make predictions and evaluate comprehensively
preds_final = final_model.predict(dtest_final)
preds_final = np.maximum(preds_final, 1e-10)  # Ensure positive predictions

# Calculate all metrics
rmse_final = np.sqrt(mean_squared_error(y_test, preds_final))
mae_final = mean_absolute_error(y_test, preds_final)

# Poisson-specific metrics
log_likelihood_final = poisson_log_likelihood(y_test, preds_final)
mean_log_likelihood_final = log_likelihood_final / len(y_test)
deviance_final = poisson_deviance(y_test, preds_final)

# AIC calculation
n_params = len(X_train.columns) + 1 if hasattr(X_train, 'columns') else 50  # Approximate
aic_final = 2 * n_params - 2 * log_likelihood_final

print(f"\n" + "="*50)
print(f"FINAL MODEL PERFORMANCE")
print(f"="*50)

print(f"\nTraditional Metrics:")
print(f"  RMSE: {rmse_final:.4f}")
print(f"  MAE: {mae_final:.4f}")

print(f"\nPoisson-Specific Metrics (PRIMARY):")
print(f"  Log-likelihood: {log_likelihood_final:.2f}")
print(f"  Mean log-likelihood: {mean_log_likelihood_final:.4f}")
print(f"  Poisson deviance: {deviance_final:.2f}")
print(f"  AIC: {aic_final:.2f}")

print(f"\nPrediction Summary:")
print(f"  Mean prediction: {preds_final.mean():.3f}")
print(f"  Mean actual: {y_test.mean():.3f}")
print(f"  Prediction std: {preds_final.std():.3f}")
print(f"  Actual std: {y_test.std():.3f}")
print(f"  Min prediction: {preds_final.min():.3f}")
print(f"  Max prediction: {preds_final.max():.3f}")

# Save the trained model with metadata
model_filename = "best_xgb_model_prem_simple.json"
final_model.save_model(model_filename)


Training final model with optimized parameters...
Training samples: 4673
Test samples: 662
Features: 49
[0]	train-rmse:1.28047	eval-rmse:1.25151
[10]	train-rmse:1.23767	eval-rmse:1.22626
[20]	train-rmse:1.20726	eval-rmse:1.20812
[30]	train-rmse:1.18419	eval-rmse:1.19689
[40]	train-rmse:1.16705	eval-rmse:1.19153
[50]	train-rmse:1.15359	eval-rmse:1.18684
[60]	train-rmse:1.14300	eval-rmse:1.18315
[70]	train-rmse:1.13327	eval-rmse:1.18160
[80]	train-rmse:1.12435	eval-rmse:1.18146
[90]	train-rmse:1.11601	eval-rmse:1.18184
[92]	train-rmse:1.11441	eval-rmse:1.18197

Actual boosting rounds used: 74

FINAL MODEL PERFORMANCE

Traditional Metrics:
  RMSE: 1.1821
  MAE: 0.9268

Poisson-Specific Metrics (PRIMARY):
  Log-likelihood: -987.02
  Mean log-likelihood: -1.4910
  Poisson deviance: 721.21
  AIC: 2074.04

Prediction Summary:
  Mean prediction: 1.508
  Mean actual: 1.486
  Prediction std: 0.448
  Actual std: 1.254
  Min prediction: 0.629
  Max prediction: 2.800


In [26]:
print("\nTop 10 Feature Importance (Gain):")
importance = final_model.get_score(importance_type='gain')
importance_df = pd.DataFrame({
    'feature': list(importance.keys()),
    'importance': list(importance.values())
}).sort_values('importance', ascending=False)

print(importance_df.head(15).to_string(index=False))


Top 10 Feature Importance (Gain):
                                                  feature  importance
                                                 team_elo   53.770771
             team_rolling_possession_touches_att_pen_area   30.693373
                             team_rolling_summary_touches   26.472380
                                team_rolling_summary_npxg   26.393230
                                  team_rolling_summary_xg   25.667360
                                             opp_team_elo   20.035305
                      team_rolling_conceded_summary_shots   18.208494
                  opp_team_rolling_conceded_summary_shots   17.924023
                                                  is_home   17.222828
                         opp_team_rolling_summary_touches   16.817179
opp_team_rolling_conceded_possession_touches_att_pen_area   16.672750
                                 team_rolling_keeper_psxg   15.075330
                   opp_team_rolling_conceded_summary_np